<a href="https://colab.research.google.com/github/AgarwalArsh11/ClauseIQ/blob/main/ClauseIQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q pdfplumber faiss-cpu sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 938.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

In [4]:
from google.colab import files

uploaded = files.upload()  # Upload your legal document PDF
pdf_path = list(uploaded.keys())[0]

Saving ICIHLIP22012V012223.pdf to ICIHLIP22012V012223.pdf
Saving HDFHLIP23024V072223.pdf to HDFHLIP23024V072223.pdf
Saving EDLHLGA23009V012223.pdf to EDLHLGA23009V012223.pdf
Saving CHOTGDP23004V012223.pdf to CHOTGDP23004V012223.pdf
Saving BAJHLIP23020V012223.pdf to BAJHLIP23020V012223.pdf


In [7]:
import pdfplumber

def extract_text_from_pdf(path):
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

raw_text = extract_text_from_pdf(pdf_path)
print(raw_text[:1000])

For Buy/ Renew/ Service/ Claim related queries Log on to www.icicilombard.com or call 1800 2666
Golden Shield
Policy Wording
b. Preamble theatre where surgical procedures are to be
carried out;
This Policy has been issued on the basis of the Disclosure to
information Norm, including the information provided by iv. Maintaining daily records of the patients and
Proposer in respect of the Insured Persons in the Proposal making them accessible to the insurance
Form, any application for insurance cover in respect of any company’s authorized representative
Insured Person and any other information or details
AYUSH Day Care Centre means and includes
submitted in relation to the Proposal Form. This Policy is a
Community Health Centre (CHC), Primary Health
contract of insurance between You and Us which is subject
Centre (PHC), Dispensary, Clinic, Polyclinic or any such
to the receipt of premium in full and accepted by Us in
health centre which is registered with the local
respect of the Insured 

In [8]:
def split_text(text, max_length=300, overlap=50):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i + max_length]
        chunks.append(" ".join(chunk))
        i += max_length - overlap
    return chunks

chunks = split_text(raw_text)
print(f"Total chunks: {len(chunks)}")


Total chunks: 86


In [9]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')  # Free, fast model
embeddings = model.encode(chunks)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [10]:
import faiss

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

In [11]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


In [1]:
def ask_question(query, k=5):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k)
    retrieved_chunks = [chunks[i] for i in I[0]]

    # Combine top-k chunks into one context
    context = " ".join(retrieved_chunks)

    # Run QA model
    result = qa_pipeline(question=query, context=context)
    return result["answer"]


In [12]:
questions = [
    "What is AYUSH Day Care Centre?",
    "What is excluded from the policy?",
    "What does the insurer cover?",
    "When can a claim be rejected?",
]

for q in questions:
    print("Q:", q)
    print("A:", ask_question(q))
    print("-" * 60)

Q: What is AYUSH Day Care Centre?
A: a Community Health Centre
------------------------------------------------------------
Q: What is excluded from the policy?
A: insurance claim
------------------------------------------------------------
Q: What does the insurer cover?
A: the same risk
------------------------------------------------------------
Q: When can a claim be rejected?
A: within 30 days
------------------------------------------------------------
